In [3]:
import pandas as pd
import os

In [1]:
ROOT_FOLDER = r"/home/tyra/Documents/CERES/resources"
ROOT_FOLDER = r"C:\Users\Orion\Documents\Projets\CERES\PMA"

### Merge CSVs 

In [4]:
nb_files = 3
li = []
for i in range(nb_files):
    path = os.path.join(ROOT_FOLDER, f'PMA_clean{i+1}.csv')
    print(path)
    dftemps = pd.read_csv(path, sep=';', index_col=None, header=0)
    li.append(dftemps)
df = pd.concat(li)

C:\Users\Orion\Documents\Projets\CERES\PMA\PMA_clean1.csv
C:\Users\Orion\Documents\Projets\CERES\PMA\PMA_clean2.csv
C:\Users\Orion\Documents\Projets\CERES\PMA\PMA_clean3.csv


In [6]:
df.to_csv(os.path.join(ROOT_FOLDER, 'PMA_full.csv'), sep=";")

### Count number of tweets for each user

In [17]:
df.groupby(['from_user_id', 'from_user_name']).size().reset_index(name='counts').sort_values('counts', ascending=False)[:15].to_csv('top_15.csv')

### Count number of mentions 

In [8]:
import re
def count_occurences(df, exp, exp_regex):
    def count_mentions(li, res=None):
        for el in li:
            res[el] = res.get(el, 0) + 1
        
    def find_mentions(value):
        reg = re.compile(exp_regex)
        return reg.findall(value)
    
    contains_mentions = df['text'].str.contains(exp)
    df_mentions = df[contains_mentions]
    
    contains_rt = df_mentions['text'].str.contains(r'RT')
    df_mentions_no_rt = df_mentions[~contains_rt]

    mentions_list = df_mentions['text'].apply(find_mentions)
    mentions_list_no_rt = df_mentions_no_rt['text'].apply(find_mentions)
    
    dic_mentions = {}
    dic_mentions_no_rt = {}
    
    mentions_list.apply(count_mentions, res=dic_mentions)
    mentions_list_no_rt.apply(count_mentions, res=dic_mentions_no_rt)
    
    final = pd.DataFrame(columns=['username', 'count'])
    final['username'] = dic_mentions.keys()
    final['count'] = dic_mentions.values()
    
    final_no_rt = pd.DataFrame(columns=['username', 'count'])
    final_no_rt['username'] = dic_mentions_no_rt.keys()
    final_no_rt['count'] = dic_mentions_no_rt.values()
    
    final = final.merge(final_no_rt, on='username')
    final = final.rename(columns={'count_x': 'total_count', 'count_y': 'count_without_RT'})
    final['pourcentage_vraies_mentions'] = round(final['count_without_RT'] / final['total_count'] * 100, 1)
    final.sort_values('total_count', ascending=False)
    
    return final

In [9]:
res = count_occurences(df, r'#', r'#\w+')
res.head()

,username,count_with_RT,count_without_RT,pourcentage_vraies_mentions
0,#PMApourToutes,78714,8252,10.5
1,#PJLbioethique,19037,1611,8.5
2,#LCI,1355,372,27.5
3,#PMA,170113,19980,11.7
4,#PMAsansPere,68015,7351,10.8


In [14]:
res = res.rename(columns={'count_with_RT': 'total_count', 'pourcentage_vraies_mentions': 'pourcentage_hashtag_sans_rt'})
res

,username,total_count,count_without_RT,pourcentage_hash_tag_sans_rt
0,#PMApourToutes,78714,8252,10.5
1,#PJLbioethique,19037,1611,8.5
2,#LCI,1355,372,27.5
3,#PMA,170113,19980,11.7
4,#PMAsansPere,68015,7351,10.8
...,...,...,...,...
38079,#debonmatin,1,1,100.0
38080,#equipeautop,1,1,100.0
38081,#Kitaout,2,1,50.0
38082,#manifs28août,1,1,100.0


In [15]:
res.to_csv(os.path.join(ROOT_FOLDER, 'hashtags.csv'), index=False, sep=";")

In [18]:
contains_mentions = df['text'].str.contains(r'@')


In [20]:
df_mentions = df[contains_mentions]

In [21]:
contains_rt = df_mentions['text'].str.contains(r'RT')

In [23]:
df_mentions_no_rt = df_mentions[~contains_rt]

In [46]:
import re
def find_mentions(value):
    reg = re.compile(r"@\w+")
    return reg.findall(value)

In [78]:
dic = {}

In [79]:
def count_mentions(li):
    for el in li:
        dic[el] = dic.get(el, 0) + 1

In [80]:
mentions_list = df_mentions['text'].apply(find_mentions)

In [81]:
mentions_list.apply(count_mentions)

0         None
1         None
2         None
3         None
4         None
          ... 
488504    None
488505    None
488508    None
488511    None
488512    None
Name: text, Length: 2039698, dtype: object

In [82]:
final = pd.DataFrame(columns=['username', 'count'])
final['username'] = dic.keys()
final['count'] = dic.values()

In [73]:
df2 = pd.read_csv(os.path.join(ROOT_FOLDER, 'mentions.csv'), sep=";")

In [83]:
len(df2["count"])

104149

In [84]:
len(final['count'])

127993

In [92]:
final = final.merge(df2, on='username')

In [99]:
final = final.rename(columns={'count_x': 'count_with_RT', 'count_y': 'count_without_RT'})

In [106]:
final['pourcentage_vraies_mentions'] = round(final['count_without_RT'] / final['count_with_RT'] * 100, 1)

In [108]:
final.sort_values('count_with_RT', ascending=False).to_csv(os.path.join(ROOT_FOLDER, 'mentions.csv'), index=False, sep=";")

In [27]:
t = s.split('\n')
ss = t[305: 307]